In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.utils import check_random_state
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_similarity
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats
plt.rcParams['font.family'] = 'NanumGothic'

df = pd.read_csv("../data/all_preprocessing_finish_without_entity.csv", encoding = "euc-kr")
df.drop("자본과부채총계", axis = 1, inplace = True)
idx = df.iloc[:,9:].isnull().sum().sort_values().head(11).index
idx2 = df.iloc[:,:9].columns
use_df = df[list(idx2) + list(idx)]
for col in use_df.iloc[:,9:].columns:
    print(f"{col}의 0을 가진 행의 갯수 : {use_df[use_df[col] == 0].shape[0]}")
    
# 금융부채, 유형자산, 금융자산, 비유동자산, 대손충당금
use_df.drop(["금융부채", "유형자산", "금융자산",  "대손충당금"], axis = 1, inplace = True)

use_df.columns[9:]

자본총계의 0을 가진 행의 갯수 : 0
자산총계의 0을 가진 행의 갯수 : 0
금융부채의 0을 가진 행의 갯수 : 544
유동부채의 0을 가진 행의 갯수 : 0
무형자산의 0을 가진 행의 갯수 : 117
대손충당금의 0을 가진 행의 갯수 : 11117
금융자산의 0을 가진 행의 갯수 : 185
유형자산의 0을 가진 행의 갯수 : 16
현금및현금성자산의 0을 가진 행의 갯수 : 0
유동자산의 0을 가진 행의 갯수 : 0
부채총계의 0을 가진 행의 갯수 : 0


C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Index(['자본총계', '자산총계', '유동부채', '무형자산', '현금및현금성자산', '유동자산', '부채총계'], dtype='object')

In [90]:
# 회사 고유값
company = df["회사명"].unique()
lst_idx = []

# 가장 최신분기에 해당하는 값 뽑기
for name in company:
    
    day = list(use_df[use_df["회사명"] == name]["결산기준일"].sort_values())[-1]
    company_index = list(use_df[(use_df["회사명"] == name) & (use_df["결산기준일"] == day)].index)[0]
    lst_idx.append(company_index)

recom_df = use_df.iloc[lst_idx, :]
recom_df.reset_index(drop = True, inplace = True)

# 코사인 값 구하기
cosine = cosine_similarity(recom_df.iloc[:,9:], recom_df.iloc[:,9:]).argsort()[:,::-1]


def get_recommend_list(df, name,  top = 30):
    company_index = recom_df[recom_df["회사명"] == name].index.values
    
    sim_index = cosine[company_index, :top].reshape(-1)
    sim_index = sim_index[sim_index != company_index]
    
    result = recom_df.iloc[sim_index, :].sort_values("자본총계", ascending = False)
    return result

a = get_recommend_list(recom_df, "CJ")